Python script offline

In [14]:
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize

In [55]:
url = 'http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?date=2000:2020&format=json&per_page=500'
resp = requests.get(url)
df_json = json.loads(resp.text)[1]
#json_normalize(df_json[1])
df_json

[{'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'},
  'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
  'countryiso3code': 'AFE',
  'date': '2020',
  'value': 677243299,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'},
  'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
  'countryiso3code': 'AFE',
  'date': '2019',
  'value': 660046272,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'},
  'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
  'countryiso3code': 'AFE',
  'date': '2018',
  'value': 643090131,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'},
  'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
  'countryiso3code': 'AFE',
  'date': '2017',
  'value': 626392880,
  'unit': '',
  'obs_status': '',
 

In [57]:
type(df_json[0])
variable_name = df_json[1]['indicator']['value']
variable_name

'Population, total'

In [95]:
def get_worldbank_api(variable, year_from='1950',year_to='2020'):
    url ="http://api.worldbank.org/v2/country/all/indicator/{}?date={}:{}&format=json&per_page=30000".format(variable,year_from,year_to)
    resp = requests.get(url)
    df_json = json.loads(resp.text)[1]
    variable_name = df_json[1]['indicator']['value']
    df = pd.json_normalize(df_json)
    df.rename(columns = {'value':variable_name}, inplace =True)
    return df.iloc[:,0:3]

###Variables and codes 
SP.POP.TOTL - population
EG.ELC.ACCS.ZS - Access to electricity (% of population)


In [97]:
get_worldbank_api("SP.POP.TOTL","2000","2001")

,countryiso3code,date,"Population, total"
0,AFE,2001,408522129.0
1,AFE,2000,398113044.0
2,AFW,2001,274433894.0
3,AFW,2000,267214544.0
4,ARB,2001,288432153.0
...,...,...,...
527,YEM,2000,17409071.0
528,ZMB,2001,10692197.0
529,ZMB,2000,10415942.0
530,ZWE,2001,11923906.0


In [115]:
df = get_worldbank_api("EG.ELC.ACCS.ZS")

In [120]:
df = pd.DataFrame({'countryiso3code':[],'date':[]})

In [121]:
df

,countryiso3code,date


In [116]:
variables = ['EG.ELC.ACCS.ZS','EG.FEC.RNEW.ZS','SP.POP.TOTL']

In [117]:
df.head()

,countryiso3code,date,Access to electricity (% of population)
0,AFE,2020,NaN
1,AFE,2019,43.640661
2,AFE,2018,42.168241
3,AFE,2017,39.754201
4,AFE,2016,38.380433


In [122]:
for variable in variables:
    df = df.merge(get_worldbank_api(variable,"2000","2020"),how='outer', on=['countryiso3code','date'])

In [123]:
df

,countryiso3code,date,Access to electricity (% of population),Renewable energy consumption (% of total final energy consumption),"Population, total"
0,AFE,2020,NaN,NaN,677243299.0
1,AFE,2019,43.640661,NaN,660046272.0
2,AFE,2018,42.168241,NaN,643090131.0
3,AFE,2017,39.754201,NaN,626392880.0
4,AFE,2016,38.380433,NaN,609978946.0
...,...,...,...,...,...
8101,ZWE,2004,35.249660,81.583900,12019911.0
8102,ZWE,2003,34.914707,77.828796,11982219.0
8103,ZWE,2002,34.200000,74.296600,11954293.0
8104,ZWE,2001,34.214050,71.531998,11923906.0
